`11_chatbot`

# Build Chatbot
- https://python.langchain.com/docs/tutorials/chatbot/

In [2]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()

True

In [4]:
from langchain_core.messages import HumanMessage, AIMessage

messages = [
    HumanMessage(content='Hi!, I am bob'),
    AIMessage(content='Hello bob. how can I help you'),
    HumanMessage(content='Say my name.')
]

llm = ChatOpenAI(model='gpt-4.1-nano')

res = llm.invoke(messages)

res.pretty_print()

================================== Ai Message ==================================

Hello, Bob!


In [5]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, END, MessagesState, StateGraph

# Graph Builder
builder = StateGraph(state_schema=MessagesState)

# Node
def simple_node(state: MessagesState):
    res = llm.invoke(state['messages'])
    return {'messages': res}

builder.add_node('simple_node', simple_node)

# Edge (Node끼리 연결)
builder.add_edge(START, 'simple_node')
builder.add_edge('simple_node', END)

# Memory (대화내역 기록)
memory = MemorySaver()

# Graph (그래프 생성)
graph = builder.compile(checkpointer=memory)

In [7]:
# 설정(conf, config, configuration -> 설정)
config = {'configurable': {'thread_id': 'abc123'}}  # 채팅방 아이디

graph.invoke({'messages': messages}, config=config)

{'messages': [HumanMessage(content='Hi!, I am bob', additional_kwargs={}, response_metadata={}, id='96ee1169-de6c-44b6-b542-1e9bcaea121f'),
  AIMessage(content='Hello bob. how can I help you', additional_kwargs={}, response_metadata={}, id='8df47862-3703-4912-8849-f543147444c2'),
  HumanMessage(content='Say my name.', additional_kwargs={}, response_metadata={}, id='48fc8425-2d0c-4db6-85cb-7de947350ab6'),
  AIMessage(content='Hi, Bob!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 4, 'prompt_tokens': 32, 'total_tokens': 36, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_7c233bf9d1', 'id': 'chatcmpl-CDKSXAFkHzKecBMNXNzmykHGqPRvZ', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--ebe7b729-

In [9]:
import uuid

u_id = uuid.uuid1()
print(u_id)

config = {'configurable': {'thread_id': '가나다123'}}  # 채팅방 아이디 -> 추후에는 UUID 형식으로 생성
messages = [
    HumanMessage(content='say my name.')
]
graph.invoke({'messages': messages}, config=config)

9d012a31-8c4f-11f0-acd2-c809a806b4c6


{'messages': [HumanMessage(content='say my name.', additional_kwargs={}, response_metadata={}, id='2c7489b7-9147-453d-b234-5827a72554d5'),
  AIMessage(content="I'm sorry, but I don't have access to your name. Could you please tell me?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 11, 'total_tokens': 29, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_7c233bf9d1', 'id': 'chatcmpl-CDKUd2jO2OM6uVdgkVsEOS45MSMk3', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--3df50675-1806-4c02-9ba1-4c0d2c17f694-0', usage_metadata={'input_tokens': 11, 'output_tokens': 18, 'total_tokens': 29, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'r

## Langgraph + `PromptTemplate`

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages([
    ('system', '너는 유능한 어시스턴트야. 너의 능력을 최대한 활용해서 답을 해봐.'),
    MessagesPlaceholder(variable_name='messages')  # 모든 저장된 대화 내용 (최신것 포함)
])

for msg in prompt_template.invoke({'messages': ['hi']}).messages:
    print(msg)

content='너는 유능한 어시스턴트야. 너의 능력을 최대한 활용해서 답을 해봐.' additional_kwargs={} response_metadata={}
content='hi' additional_kwargs={} response_metadata={}


## State 확장

In [26]:
# 내장된 MessagesState를 확장해서 사용
class MyState(MessagesState):
    # 상속받아서 이미 key 'messages'는 있음
    # messages: Annotated[list[AnyMessage], add_messages]
    lang: str

builder = StateGraph(state_schema=MyState)

prompt_template = ChatPromptTemplate.from_messages([
    ('system', '너는 유능한 어시스턴트야. 너의 능력을 최대한 활용해서 답을 해봐. {lang} 언어로 답해.'),
    MessagesPlaceholder(variable_name='messages')  # 모든 저장된 대화 내용(최신것 포함)
])

def simple_node(state: MyState):
    # prompt 추가.
    chain = prompt_template | llm  # 체인 방식
    res = chain.invoke(state)

    return {'messages': res}

builder.add_node('simple_node', simple_node)

builder.add_edge(START, 'simple_node')
builder.add_edge('simple_node', END)

memory = MemorySaver()

graph = builder.compile(checkpointer=memory)

In [27]:
config = {'configurable': {'thread_id': 'abc1'}} 
state = {
    'messages': [HumanMessage(content='Hi. I am bob')],
    'lang': 'Spanish'
}

res = graph.invoke(state, config)

for msg in res['messages']:
    msg.pretty_print()

================================ Human Message =================================

Hi. I am bob
================================== Ai Message ==================================

¡Hola, Bob! Encantado de conocerte. ¿En qué puedo ayudarte hoy?


## 대화 기록 관리하기
대화 내역을 관리하지 않으면, LLM의 컨텍스트 윈도우(입력 최대치)를 넘어가 버림.

In [34]:
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage, trim_messages

# trim - 정리하다
trimmer = trim_messages(
    strategy='last',  # 최신 메시지들을
    max_tokens=65,  # 최대 65토큰 까지만 허용,
    token_counter=llm,  # llm 모델에 맞춰서 토큰 세고
    include_system=True,  # System 프롬프트는 포함(정리X)
    allow_partial=False,  # 메세지 중간에서 자르지는 말고
    start_on='human'  # 잘린 메세지의 첫번째는 사람 메세지가 되도록
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

In [47]:
class MyState(MessagesState):
    # messages: ~~
    lang: str


builder = StateGraph(state_schema=MyState)

prompt_template = ChatPromptTemplate.from_messages([
    ('system', '너는 유능한 어시스턴트야. 너의 능력을 최대한 활용해서 답을 해봐. {lang} 언어로 답해.'),
    MessagesPlaceholder(variable_name='messages')  # 모든 저장된 대화 내용(최신것 포함)
])


def simple_node(state: MyState):
    # 메세지 정리 -> 프롬프트 생성 -> LLM 답변
    # print('정리 전 메시지 개수: ', len(state['messages']))
    trimmed_messages = trimmer.invoke(state['messages'])
    # print('정리 후 메시지 개수: ', len(trimmed_messages))
    
    # 체인 생성
    chain = prompt_template | llm
    
    # 정리된 메세지로 state 교체 후, 체인 실행
    state['messages'] = trimmed_messages
    res = chain.invoke(state)

    return {'messages': res}


builder.add_node('simple_node', simple_node)

builder.add_edge(START, 'simple_node')
builder.add_edge('simple_node', END)

memory = MemorySaver()

graph = builder.compile(checkpointer=memory)

In [50]:
config = {'configurable': {'thread_id': 'abc1'}} 
state = {
    'messages': [HumanMessage(content='내 상태는 어때')],
    'lang': '한국어'
}

for chunk, metadata in graph.stream(state, config, stream_mode='messages'):
    print(chunk.content, end='|')

|제가| 직접| 당신|의| 상태|를| 알| 수|는| 없|지만|,| 건강|이나| 기|분|이| 걱|정|된다|면| 다음|과| 같은| 질문|을| 스|스로| 해|보|는| 것이| 도움이| 될| 수| 있어|요|:

|-| 지금| 몸|이나| 마음|이| 어떤| 기|분|인가|요|?
|-| 특별|히| 아|프|거나| 불|편|한| 곳|이| 있|나요|?
|-| 최근|에| 어떤| 변화|나| 힘|든| 일이| 있었|나요|?
|-| 충분|히| 휴|식을| 취|하고| 있|나요|?
|-| 필요한| 도움|이나| 상담|이| 필요|하|다고| 느|끼|시|나요|?

|혹|시| 구|체|적으로| 걱|정|되|거나| 어려|운| 점|이| 있다|면| 더| 자세|히| 알려|주|시면| 도|와|드|릴| 수| 있어|요|.| 항상| 건강|과| 안전|이| 최|우|선|입니다|!||